In [ ]:
from gurobipy import *

In [26]:
try:

    plants = ['PlantOne','PlantTwo']
    
    tsc = ['A','B']
    
    regions = ['RegionOne','RegionTwo','RegionThree']

    incost = {
        ('PlantOne','A'): 190,
        ('PlantOne','B'): 210,
        ('PlantTwo','A'): 185,
        ('PlantTwo','B'): 205
    }
    outcost = {
        ('A','RegionOne'): 175,
        ('A','RegionTwo'): 180,
        ('A','RegionThree'): 165,
        ('B','RegionOne'): 235,
        ('B','RegionTwo'): 130,
        ('B','RegionThree'): 145,
    }
    supply = {
        ('PlantOne'): 100,
        ('PlantTwo'): 125
    }
    tcap = {
        ('A'): 5000,
        ('B'): 5000
    }
    demand = {
        ('RegionOne'): 25,
        ('RegionTwo'): 95,
        ('RegionThree'): 80
    }

    # Create a new model
    m = Model("transport_problem_1")

    # Create variables
    inflow = {}
    outflow = {}
    for i in plants:
        for j in tsc:
            inflow[i,j] = m.addVar(obj=incost[i,j], name='inflow_%s_%s' % (i, j))
    for j in tsc:
        for k in regions:
            outflow[j,k] = m.addVar(obj=outcost[j,k],name='outflow_%s_%s' % (j, k))
    # Integrate new variables
    m.update()

    # Add supply constraints
    for i in plants:
        m.addConstr(quicksum(inflow[i,j] for j in tsc) <= supply[i], 'supply_%s' % (i))
    
    # Add demand constraints
    for k in regions:
        m.addConstr(quicksum(outflow[j,k] for j in tsc) >= demand[k], 'demand_%s' % (k))
    
    # Add capacity constraints for TS centers
    for j in tsc:
        m.addConstr(quicksum(inflow[i,j] for i in plants) <= tcap[j], 'tcap_%s' % (j))
    
    
    #Add flow balance constraints cap[%s,%s]" % (i, j)
    
    for j in tsc:
        m.addConstr(quicksum(inflow[i,j] for i in plants) == quicksum(outflow[j,k] for k in regions), 'flow_%s,%s' % (i,k))
   

    m.setObjective(quicksum(incost[i,j]*inflow[i,j] for i in plants for j in tsc) + quicksum(outcost[j,k]*outflow[j,k] for j in tsc for k in regions))
    # Optimize the model. The default ModelSense is to is to minimize the objective, which is what we want.
    m.optimize()
    

    # Print solution
    if m.status == GRB.status.OPTIMAL:
        print ('\nOptimal inflows :')
        for i in plants:
            for j in tsc:
                print (i, '->', j, ':', inflow[i,j].x)
    
    if m.status == GRB.status.OPTIMAL:
        print ('\nOptimal outflows :')
        for j in tsc:
            for k in regions:
                print (j, '->', k, ':', outflow[j,k].x)


except GurobiError:
    print ('Error reported')

Optimize a model with 9 rows, 10 columns and 24 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 5e+03]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 7 rows, 10 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.500000e+01   0.000000e+00      0s
       7    6.9200000e+04   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.02 seconds
Optimal objective  6.920000000e+04

Optimal inflows :
PlantOne -> A : 75.0
PlantOne -> B : 0.0
PlantTwo -> A : 0.0
PlantTwo -> B : 125.0

Optimal outflows :
A -> RegionOne : 25.0
A -> RegionTwo : 0.0
A -> RegionThree : 50.0
B -> RegionOne : 0.0
B -> RegionTwo : 95.0
B -> RegionThree : 30.0
